# --- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:
```
$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?
```
You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:
```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

`cd` means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
- `cd x` moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
- `cd ..` moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
- `cd /` switches the current directory to the outermost directory, /.
`ls` means list. It prints out all of the files and directories immediately contained by the current directory:
- `123 abc` means that the current directory contains a file named abc with size 123.
- `dir xyz` means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:
```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
Directory d has total size 24933642.
As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [ ]:
import logging
import sys

sys.path.append("..")
from aoc import get_logger, get_data, timer, test

logger = get_logger("day07", logging.INFO)
data = get_data("day07")

logger.info("initialization complete!")

## Solution Part One
Today's puzzle required a little more thinking. To get our input into a suitable structure we'll parse it into a list of commands, each paired with their output.

In [ ]:
def parsed(input_) -> list:
    """Parse input into a list of pairs (command, [output])"""
    result = []
    command = ""
    output = []
    for line in input_.split("\n"):
        if line.startswith("$"):
            if command:
                result.append((command, output))
                output = []
            command = line[2:]
        else:
            output.append(line)
    result.append((command, output))
    return result

Next, we'll create a class which abstracts the terminal interface. Using the methods of this class we can run the commands from our input and generate the file structure needed to solve the puzzle. Note that we use the inverse of the `ls` command: Instead of outputting which files exist, it creates files based on the input of an `ls` command.

In [ ]:
class CommandLine(object):
    def __init__(self):
        self.cwd = "/"
        self.files = {}  # {path: size}
        self.dirs = []

    def cd(self, arg: str):
        """Change current working directory"""
        if arg == "/":
            self.cwd = "/"
        elif arg == "..":
            self.cwd = "/".join(self.cwd.split("/")[:-2]) + "/"
        else:
            self.cwd += f"{arg}/"

    def sl(self, output: list):
        """Inverse ls, create files and directories using the output of ls"""
        for line in output:
            if not len(line.split()) == 2:
                print(line)
            value, name = line.split()
            if value == "dir":
                self.dirs.append(f"{self.cwd}{name}")
            else:
                self.files[f"{self.cwd}/{name}"] = int(value)

We'll add a function to generate the file system from the input, using our CommandLine class and the commands from the input. We'll also add a function to get the size of any directory, using its path and the complete file system. It runs through the file system's dict of known by the files, and sums the size of the files whose file path contains the path of the directory we're checking.

In [ ]:
def build_file_system(parsed_input) -> CommandLine:
    cli = CommandLine()

    for command in parsed_input:
        cmd, *arg = command[0].split()
        if cmd == "cd":
            cli.cd(*arg)
        elif cmd == "ls":
            cli.sl(command[1])
    return cli


def get_dir_size(dir_path, cli):
    return sum([size for path, size in cli.files.items() if path.startswith(dir_path)])

To complete part 1 we parse the input and feed it to our CommandLine class using the build_file_system function. This will run all the commands from the input on it, so our cli object contains all the info on the files and dirs we need for the solution.

In [ ]:
@timer
def part1(input_):
    parsed_input = parsed(input_)
    cli = build_file_system(parsed_input)

    dir_sizes = [get_dir_size(s, cli) for s in cli.dirs]
    return sum([s for s in dir_sizes if s <= 100000])


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 1:", part1(data.input))

# --- Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:
```
Delete directory e, which would increase unused space by 584.
Delete directory a, which would increase unused space by 94853.
Delete directory d, which would increase unused space by 24933642.
Delete directory /, which would increase unused space by 48381165.
````
Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

## Solution Part Two
Similar to part 1, but with some more variables.

In [ ]:
@timer
def part2(input_):
    parsed_input = parsed(input_)
    cli = build_file_system(parsed_input)
    space_used = get_dir_size("/", cli)
    unused_space = 70000000 - space_used
    space_needed = 30000000 - unused_space

    dir_sizes = [get_dir_size(d, cli) for d in cli.dirs]
    return min([s for s in dir_sizes if s >= space_needed])


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 2:", part2(data.input))